# Functions

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.io import loadmat
import os
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

In [3]:
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

In [7]:
exp_folder = 'D:\\GoogleDrive\\retina\\Troy\'s data\\20210908\\SplitData\\'
channelnumber = 30

dt = 0.01

for filename in os.listdir(exp_folder):
    if filename.find('diode_WhiteNoise_mean=10_amp=2_11-Aug-2021_0')==-1:
        continue
    print(filename[:-17]+filename[-5])
    annots = loadmat(exp_folder+filename)
    sampling_rate = 20000
    TimeStamps = np.squeeze(annots['TimeStamps'])
    x = annots['a_data'][0, int(TimeStamps[0]*sampling_rate):int(TimeStamps[1]*sampling_rate)+1]
    x = x[::int(sampling_rate*dt)]
    x = x.astype(float)
    x = (x-np.mean(x))/np.std(x)
    T=np.arange(len(x))*dt
    # x_list.append(x.copy())
    rstate, _ = np.histogram(annots['Spikes'][0][channelnumber-1]-TimeStamps[0], np.append(0,T))

diode_WhiteNoise_mean=10_amp=2_0


In [8]:
def STcollection(r, x, dt, window):
    negshift = round(window[0]/dt) # second
    posshift = round(window[1]/dt) # second
    xcoll = np.zeros(posshift-negshift)
    for i in np.arange(-negshift, len(r)-posshift):
        counter = r[i]
        while counter != 0:
            xcoll = np.vstack((xcoll , x[i+negshift:i+posshift]))
            counter -= 1
    time = np.arange(negshift*dt, posshift*dt, dt)
    return time, xcoll[1:]

In [11]:
timeshift, xcoll = STcollection(rstate, x, dt, [-1,1])

In [19]:
pca=PCA(n_components=2)
pca.fit(xcoll).transform(xcoll)

array([[-2.81815649, -2.20960396],
       [-3.44229922, -2.26071143],
       [-3.44229922, -2.26071143],
       ...,
       [-3.80939915, -2.73956475],
       [-3.84760806, -2.34717883],
       [-3.71569983, -1.85321012]])

In [21]:
pca.explained_variance_ratio_

array([0.05602781, 0.0320709 ])